## Mask Wearing Detection - Transfer Learning on SOTA ScaledYOLOv4 model

* Download mask detection data
* Setup ScaledYOLOv4 Training configuration
* Train model
* Evaluate model training performance
* Visualize annotated training data
* Model inference on test data
* Visualize model inference results
* Evaluate model performance on test data



### Install Model Dependencies

In [ ]:
!git clone -b yolov4-large https://github.com/roboflow-ai/ScaledYOLOv4.git  # clone the yolov4-large branch of the open-source ScaledYOLOv4 repo
# Scaled YOLOv4: Scaling Cross Stage Partial Network - https://arxiv.org/abs/2011.08036
%cd /content/ScaledYOLOv4/

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 338.47 KiB | 21.15 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/ScaledYOLOv4


In [ ]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.8.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
%cd /content/
!git clone https://github.com/JunnYu/mish-cuda  #install mish activation funciton for cuda GPU
# Mish is a novel self-regularized non-monotonic activation function: f(x) = x tanh(softplus(x)); outperformed ReLU and Leaky ReLU - https://arxiv.org/pdf/1908.08681.pdf
%cd mish-cuda
!python setup.py build install

/content
Cloning into 'mish-cuda'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 17.40 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/mish-cuda
/usr/lib/python3.7/distutils/extension.py:131: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-3.7/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manifest file 'src/mish_cu

In [ ]:
!pip install -U PyYAML  # PyYAML enables reading the data file

     |████████████████████████████████| 645kB 19.9MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd /content/ScaledYOLOv4/


/content/ScaledYOLOv4


### Download Annotated Dataset 

- 1,815 640x640 Preprocessed images (required model input resolution) before augmentation
- 70% Training data (3,804 including augmentation), 20% validation (362 images) and 10% test data (183 images)

In [ ]:
%cd /content

!curl -L "https://app.roboflow.com/ds/wIBsLFZkvX?key=huvnv6PNyz" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# download dataset preprocessed with resizing, our Bremner team's manuel annotations, plus augmentations in Roboflow

Streaming output truncated to the last 5000 lines.
 extracting: train/images/986_jpg.rf.2de54ea32ec8ab87c25fc7c2776ee50b.jpg  
 extracting: train/images/986_jpg.rf.55b7acaf1e386e924bf3b02c99ef36f8.jpg  
 extracting: train/images/986_jpg.rf.e25cca973ff2be264267ff1443c985ce.jpg  
 extracting: train/images/988_jpg.rf.0c64b6b1ea864278bc04aab8b56a9e3e.jpg  
 extracting: train/images/988_jpg.rf.1fccc6d8c4106dd809ff497a3623f539.jpg  
 extracting: train/images/988_jpg.rf.d777cf745d214679f135a0b6dbc0c78b.jpg  
 extracting: train/images/98_jpg.rf.4659d2edfe6a3a5c02da047aa4445c2b.jpg  
 extracting: train/images/98_jpg.rf.4bf11a24109d3aa7352d02814c964f81.jpg  
 extracting: train/images/98_jpg.rf.592857399daa89c7cd065a1787645783.jpg  
 extracting: train/images/991_jpeg.rf.1fad1f629c09d20fa039312b8108a9cb.jpg  
 extracting: train/images/991_jpeg.rf.2641ee237d252056f355e06bb98d7af1.jpg  
 extracting: train/images/991_jpeg.rf.e74fe854ba0cf44e59dfa22896edb9e9.jpg  
 extracting: train/images/992_jpg.rf.

In [ ]:
%cat data.yaml # view the dataset YAML file

train: ../train/images
val: ../valid/images

nc: 2
names: ['mask', 'no-mask']